In [ ]:

from fastapi import FastAPI, Request
import mysql.connector
from pydantic import BaseModel
from typing import List, Optional, Dict
import json

app = FastAPI()

# Database connection function
def get_db_connection():
    return mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='###########',
        port='3306',
        database='chatbot'
    )

# Webhook request schema
class WebhookRequest(BaseModel):
    queryResult: dict

# Function to fetch data from a specified table
def fetch_data_from_db(query, params):
    try:
        mydb = get_db_connection()
        mycursor = mydb.cursor()
        mycursor.execute(query, params)
        result = mycursor.fetchall()
        mydb.close()
        return result
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

@app.post("/webhook")
async def dialogflow_webhook(req: WebhookRequest):
    intent = req.queryResult['intent']['displayName']
    parameters = req.queryResult.get('parameters', {})
    print(intent, parameters)

    calorie_limit = parameters.get("calorie_limit")

    table_name, item_name = None, None
    for table, value in parameters.items():
        if value:
            table_name = table
            item_name = value
            break
    print(table_name, item_name)

    if intent == "food.items":
        table_name = parameters.get("table_name", "").replace(" ", "_")
        if not table_name:
            return {"fulfillmentText": "Please specify a category to list items."}
        return await get_items_in_category(table_name)

    elif intent == "items.ingredients":
        return await get_ingredients(table_name, item_name)

    elif intent == "veggie.sandwiches":
        return await get_sandwiches_with_veggies_and_no_meat()

    elif intent == "pure.veggie.sandwiches":
        return await get_sandwiches_with_veggies_and_no_meat_and_egg()

    elif intent == "nutritional.information":
        return await get_calories(table_name, item_name)

    elif intent == "items.without.ingredient":
        ingredient = parameters.get("ingredient")
        return await get_items_without_ingredient_in_sandwiches(ingredient)

    elif intent == "drinks.without.ingredient":
        ingredient = parameters.get("ingredient")
        return await get_drinks_without_ingredient(ingredient)

    elif intent == "items.with.ingredient":
        ingredient = parameters.get("ingredient")
        return await get_items_with_ingredient_across_tables(ingredient)

    elif intent == "drinks.with.ingredient":
        ingredient = parameters.get("ingredient")
        return await get_drinks_with_ingredient(ingredient)

    elif intent == "items.without.ingredient.and.below.calories":
        ingredient = parameters.get("ingredient")
        calorie_limit = parameters.get("calorie_limit")
        return await get_items_without_ingredient_and_below_calories(ingredient, calorie_limit)

    elif intent == "items.without.meat":
        return await get_sandwiches_without_meat()

    elif intent == "item.price":
        return await get_item_cost(table_name, item_name)

    elif intent == "items.below.price":
        price = parameters.get("unit-currency", {}).get('amount')
        return await get_items_below_price_with_tax(price)

    elif intent == "items.above.price":
        price = parameters.get("unit-currency", {}).get('amount')
        return await get_items_above_price_with_tax(price)

    elif intent == "drinks.below.price":
        price = parameters.get("unit-currency", {}).get('amount')
        return await get_drinks_below_price_with_tax(price)

    elif intent == "drinks.above.price":
        price = parameters.get("unit-currency", {}).get('amount')
        return await get_drinks_above_price_with_tax(price)

    elif intent == "items.below.calories":
        calorie_limit = parameters.get("calorie_limit")
        print("Raw calorie_limit:", calorie_limit)

        if calorie_limit is None:
            return {"fulfillmentText": "Please provide a calorie limit for the search."}

        if isinstance(calorie_limit, list):
            calorie_limit = calorie_limit[0]

        try:
            calorie_limit = int(calorie_limit)
        except (ValueError, TypeError) as e:
            print("Error converting calorie_limit:", e)
            return {"fulfillmentText": "Invalid calorie limit provided."}

        print("Processed calorie_limit:", calorie_limit, type(calorie_limit))
        return await get_items_below_calories(calorie_limit)

    elif intent == "drinks.below.calories":
        calorie_limit = parameters.get("calorie_limit")
        print("Raw calorie_limit:", calorie_limit)

        if calorie_limit is None:
            return {"fulfillmentText": "Please provide a calorie limit for the search."}

        if isinstance(calorie_limit, list):
            calorie_limit = calorie_limit[0]

        try:
            calorie_limit = int(calorie_limit)
        except (ValueError, TypeError) as e:
            print("Error converting calorie_limit:", e)
            return {"fulfillmentText": "Invalid calorie limit provided."}

        print("Processed calorie_limit:", calorie_limit, type(calorie_limit))
        return await get_drinks_below_calories(calorie_limit)

    elif intent == "items.above.calories":
        calorie_limit = parameters.get("calorie_limit")
        print("Raw calorie_limit:", calorie_limit)

        if calorie_limit is None:
            return {"fulfillmentText": "Please provide a calorie limit for the search."}

        if isinstance(calorie_limit, list):
            calorie_limit = calorie_limit[0]

        try:
            calorie_limit = int(calorie_limit)
        except (ValueError, TypeError) as e:
            print("Error converting calorie_limit:", e)
            return {"fulfillmentText": "Invalid calorie limit provided."}

        print("Processed calorie_limit:", calorie_limit, type(calorie_limit))
        return await get_items_above_calories(calorie_limit)

    elif intent == "drinks.above.calories":
        calorie_limit = parameters.get("calorie_limit")
        print("Raw calorie_limit:", calorie_limit)

        if calorie_limit is None:
            return {"fulfillmentText": "Please provide a calorie limit for the search."}

        if isinstance(calorie_limit, list):
            calorie_limit = calorie_limit[0]

        try:
            calorie_limit = int(calorie_limit)
        except (ValueError, TypeError) as e:
            print("Error converting calorie_limit:", e)
            return {"fulfillmentText": "Invalid calorie limit provided."}

        print("Processed calorie_limit:", calorie_limit, type(calorie_limit))
        return await get_drinks_above_calories(calorie_limit)

    elif intent == "order.total.cost":
        processed_parameters = {}
        for table, value in parameters.items():
            if isinstance(value, str):
                items = [item.strip() for item in value.split(",") if item.strip()]
                if items:
                    processed_parameters[table] = items
            elif isinstance(value, list):
                items = [item.strip() for item in value if item.strip()]
                if items:
                    processed_parameters[table] = items

        print("Processed parameters:", processed_parameters)
        return await calculate_order_total(processed_parameters)

    elif intent == "get.max.min.price":
        table_name = parameters.get("table_name")
        if isinstance(table_name, list):
            table_name = table_name[0]
        table_name = table_name.replace(" ", "_") if table_name else None

        query_type = parameters.get("query_type").lower()

        if not table_name:
            return {"fulfillmentText": "Please specify a valid category to find the prices."}

        return await get_max_min_price_in_category(table_name, query_type)

    elif intent == "get.bagel.price":
        bagel_type = parameters.get("bagel_type", "")
        cream_cheese = parameters.get("cream_cheese", "")

        if not bagel_type or not cream_cheese:
            return {
                "fulfillmentText": "Please specify both the bagel type and the cream cheese to get the price."
            }

        return await get_bagel_price_with_cream_cheese(bagel_type, cream_cheese)

    elif intent == "get.bagel.type":
        sandwich_name = parameters.get("egg_sandwiches") or parameters.get("Lunch_sandwiches") or parameters.get("sandwich", "")

        if not sandwich_name:
            return {
                "fulfillmentText": "Please specify the sandwich name to get the bagel type."
            }

        return await get_bagel_type_in_sandwich(sandwich_name)

    elif intent == "heavy.calories":
        return await get_heavy_sandwiches()

    elif intent == "light.calories":
        return await get_light_sandwiches()

    return {"fulfillmentText": "Sorry, I couldn't process that request."}
